In [1]:
import sys
import os
import matplotlib.pyplot as plt
import torch
# Add the src directory to the path. TEMPORARY FIX
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../..")))

from models.deeplab.deeplab import DeepLabV3
from src.models.data_management.cnn_formes import CNNFormes
from src.data_processing.dataset_loader import CoastData
from src.models.metrics import Metrics

from src.models.data_management.data_loader import DataLoaderManager

import cv2

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [3]:
artifact_path = os.path.abspath(os.path.join(os.getcwd(), "../../../artifacts/"))

num_classes = 3
metrics = {
    "train": Metrics(phase="train", num_classes=num_classes, average=None, use_margin=False),
    "validation": Metrics(phase="val", num_classes=num_classes, average=None, use_margin=False),
    "test": Metrics(phase="test", num_classes=num_classes, average=None, use_margin=False) # 'macro'
}

# Load the data to split it and save it to a dict
model  = DeepLabV3(num_classes=3, experiment_name="test_experiments", use_mlflow=False, pretrained=False)

model.load_model(os.path.abspath(os.path.join(artifact_path, "2025-03-12-11-44-51_deeplab/models/best_model.pth")))

data_path = os.path.abspath(os.path.join(os.getcwd(), "../../../data/processed"))

coast_data = CoastData(data_path)
split = coast_data.split_data()

data = DataLoaderManager.load_data(split)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


CoastData: global - 1717 images
Coast: agrelo, Total size: 244
Coast: arenaldentem, Total size: 40
Coast: cadiz, Total size: 946
Coast: cies, Total size: 430
Coast: samarador, Total size: 57


In [5]:
for split in data:
    print(f"Split: {split}")
    for img_path, mask_path in zip(data[split]["images"], data[split]["masks"]):
        pred = model.predict_patch(img_path, combination="max")
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = torch.from_numpy(mask).float() 
        metrics[split].update_metrics(pred, mask)

    metrics[split].compute()
    print(metrics[split].get_last_epoch_info())

Split: train


/opt/conda/lib/python3.10/site-packages/torcheval/metrics/functional/classification/accuracy.py:275: UserWarning: The reduce argument of torch.scatter with Tensor src is deprecated and will be removed in a future PyTorch release. Use torch.scatter_reduce instead for more reduction options. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/TensorAdvancedIndexing.cpp:230.)
  num_correct = mask.new_zeros(num_classes).scatter_(0, target, mask, reduce="add")


train metrics: 
	train_accuracy: tensor([0.9980, 0.9849, 0.9887])
	train_f1_score: tensor([0.9932, 0.9886, 0.9886])
	train_precision: tensor([0.9884, 0.9924, 0.9885])
	train_recall: tensor([0.9980, 0.9849, 0.9887])
	train_confusion_matrix: 
		0.9980 0.0007 0.0013
		0.0048 0.9849 0.0103
		0.0041 0.0072 0.9887

Split: validation
val metrics: 
	val_accuracy: tensor([0.9989, 0.9798, 0.9816])
	val_f1_score: tensor([0.9937, 0.9826, 0.9826])
	val_precision: tensor([0.9885, 0.9854, 0.9837])
	val_recall: tensor([0.9989, 0.9798, 0.9816])
	val_confusion_matrix: 
		0.9989 0.0006 0.0005
		0.0046 0.9798 0.0156
		0.0041 0.0143 0.9816

Split: test
test metrics: 
	test_accuracy: tensor([0.9986, 0.9775, 0.9769])
	test_f1_score: tensor([0.9930, 0.9793, 0.9792])
	test_precision: tensor([0.9873, 0.9811, 0.9815])
	test_recall: tensor([0.9986, 0.9775, 0.9769])
	test_confusion_matrix: 
		0.9986 0.0008 0.0006
		0.0052 0.9775 0.0174
		0.0043 0.0188 0.9769

